# Prophet - Predecir el valor deL ORO


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

En mi caso use el oro

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2022-11-10'
start_date = '2016-01-01'
# Descargar el dataframe
aus_df = yf.download('GC=F',start_date, today).reset_index()

aus_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1720,2022-11-03,"$1,629.20","$1,629.20","$1,615.10","$1,627.30","$1,627.30",1427
1721,2022-11-04,"$1,630.20","$1,674.50","$1,629.00","$1,672.50","$1,672.50",159
1722,2022-11-07,"$1,676.90","$1,677.90","$1,672.90","$1,676.50","$1,676.50",42
1723,2022-11-08,"$1,703.20","$1,712.10","$1,703.20","$1,712.10","$1,712.10",23
1724,2022-11-09,"$1,707.40","$1,720.20","$1,706.00","$1,710.10","$1,710.10",23


In [ ]:
aus_df.isnull().any()

Date         False
Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
dtype: bool

In [ ]:
# Validamos que no hay datos vacíos
aus_df.isnull().any()


Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
aus_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
aus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725 entries, 0 to 1724
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1725 non-null   datetime64[ns]
 1   Open       1725 non-null   float64       
 2   High       1725 non-null   float64       
 3   Low        1725 non-null   float64       
 4   Close      1725 non-null   float64       
 5   Adj Close  1725 non-null   float64       
 6   Volume     1725 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 94.5 KB


In [ ]:
au_df = aus_df[['Date', 'Open']]
au_df.head()

,Date,Open
0,2016-01-04,"$1,063.40"
1,2016-01-05,"$1,075.60"
2,2016-01-06,"$1,081.60"
3,2016-01-07,"$1,091.60"
4,2016-01-08,"$1,111.10"


In [ ]:
#creacion del diccionario
new_names = {
    "Date": "ds", 
    "Open": "y",
}

In [ ]:
df =au_df
df.head()

,Date,Open
0,2016-01-04,"$1,063.40"
1,2016-01-05,"$1,075.60"
2,2016-01-06,"$1,081.60"
3,2016-01-07,"$1,091.60"
4,2016-01-08,"$1,111.10"


In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = 

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para rembrar las columnas de df
df


In [ ]:
df=df.rename(columns = new_names)

In [ ]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725 entries, 0 to 1724
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      1725 non-null   datetime64[ns]
 1   y       1725 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 27.1 KB


In [ ]:
df.tail()

,ds,y
1798,2022-10-12 00:00:00+00:00,"$1,279.73"
1799,2022-10-13 00:00:00+00:00,"$1,294.92"
1800,2022-10-14 00:00:00+00:00,"$1,288.05"
1801,2022-10-15 00:00:00+00:00,"$1,297.31"
1802,2022-10-16 00:00:00+00:00,"$1,275.01"


In [ ]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Variacion de Oro-USD"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [ ]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Variacion de Oro-USD"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [ ]:
m = Prophet(seasonality_mode = 'multiplicative' )

In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"



In [ ]:
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2085,2023-11-05
2086,2023-11-06
2087,2023-11-07
2088,2023-11-08
2089,2023-11-09


In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = 
future.tail()

,ds
2163,2023-10-12
2164,2023-10-13
2165,2023-10-14
2166,2023-10-15
2167,2023-10-16


In [ ]:
future = future[ future['ds'].dt.dayofweek < 5 ]

In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
forecast = m.predict(future)

In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = 

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1981,2023-11-03,"$1,612.95",$599.27,"$2,608.47"
1982,2023-11-06,"$1,616.39",$594.88,"$2,647.85"
1983,2023-11-07,"$1,616.27",$592.13,"$2,666.00"
1984,2023-11-08,"$1,616.78",$586.91,"$2,630.97"
1985,2023-11-09,"$1,618.35",$583.47,"$2,656.75"


In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2163,2023-10-12,"$-3,319.09","$-26,822.91","$23,403.79"
2164,2023-10-13,"$-3,441.80","$-26,927.89","$23,794.22"
2165,2023-10-14,"$-3,578.54","$-27,556.84","$23,861.05"
2166,2023-10-15,"$-3,717.96","$-28,360.17","$24,558.36"
2167,2023-10-16,"$-3,861.58","$-29,235.90","$24,712.87"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
#oro
plot_components_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)